In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchlensmaker as tlm

In [ ]:
lens_diameter = 60. # lens diameter in mm

class Optics(tlm.Module):
    def __init__(self):
        super().__init__()
        
        self.shape = tlm.CircularArc(lens_diameter, nn.Parameter(torch.tensor(-200.)))

        self.lens = tlm.PlanoLens(self.shape, (1.0, 1.5), outer_thickness=2.0)

        self.optics = nn.Sequential(
            tlm.ParallelBeamRandom(0.9*lens_diameter),
            tlm.GapY(15.),
            
            self.lens,
            
            tlm.GapY(100.0), # focal length
            tlm.FocalPointLoss(),
        )

    def forward(self, inputs):
        return self.optics(inputs)

optics = Optics()

init = (10, torch.tensor([0., 0.]))
tlm.render_plt(optics, init)

tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-3),
    inputs = init,
    num_iter = 250
)

tlm.render_plt(optics, init)

In [ ]:
part = tlm.lens_to_part(optics.lens)
part